In [ ]:
import os
import logging
import pandas as pd
from datetime import datetime
from typing import Tuple, List, Optional
from etherscan_functions import get_erc20_transfers, get_block_numbers_by_date
'''
不用看，测试文件
'''

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger()

START_DATE = "2024-12-10 08:00"
END_DATE = "2024-12-12 08:00" # so that when cenverting to UTC, it will be 00:00
OUTPUT_FILE = 'C:/Users/YuweiCao/Documents/GitHub/Project/Project/etherscan/result'
api_key = "VQAIR728IM4Z8RZKPYBR4ESM5I3WBZK2C1" # my free API key, you can get one at https://etherscan.io/myapikey
base_url = "https://api.etherscan.io/v2/api" # We're using the v2 API 2024/12/12
ADDRESS = "0x5be9a4959308A0D0c7bC0870E319314d8D957dBB" # Address of the contract we want to get the source code of
chain_id = 1  # Ethereum Mainnet

In [2]:
def fetch_erc20_transfers(address: str, start_block: int, end_block: int, offset: int = 100) -> pd.DataFrame:
    """
    Fetch ERC20 token transfer data and process it into a DataFrame.
    :param address: Contract address you decide to fetch the data from.
    :param start_block: Start block number.
    :param end_block: End block number.
    :param offset: Number of transactions to fetch per API call.
    """
    all_transfers = []
    page = 1

    logger.info(f"Fetching ERC20 transfers from block {start_block} to {end_block}...")

    while True:
        try:
            transfers = get_erc20_transfers(
                address=address, startblock=start_block, endblock=end_block, page=page, offset=offset
            )

            if transfers is None:
                logger.warning("API returned None. Exiting...")
                break
            if isinstance(transfers, list) and len(transfers) == 0:
                logger.info("Empty list received. Exiting...")
                break
            if isinstance(transfers, pd.DataFrame) and transfers.empty:
                logger.info("Empty DataFrame received. Exiting...")
                break

            if isinstance(transfers, list):
                transfers_df = pd.DataFrame(transfers)
            elif isinstance(transfers, pd.DataFrame):
                transfers_df = transfers
            else:
                logger.warning(f"Unexpected data format: {type(transfers)}. Exiting...")
                break

            if transfers_df.empty:
                logger.info("Empty DataFrame received. Exiting...")
                break

            all_transfers.extend(transfers_df.to_dict(orient='records'))
            logger.info(f"Page {page}: Retrieved {len(transfers_df)} transactions.")
            page += 1

        except Exception as e:
            logger.warning(f"API request failed on page {page}: {e}")
            break

    final_df = pd.DataFrame(all_transfers)
    logger.info(f"Final dataset contains {len(final_df)} unique transactions.")
    return final_df


In [3]:
def process_and_save_transfers(transfers_df: pd.DataFrame, output_file: str) -> None:
    """
    Process ERC20 transfers DataFrame and save to a CSV file.
    """
    if not transfers_df.empty:
        transfers_df['dateTime'] = pd.to_datetime(
            pd.to_numeric(transfers_df['timeStamp'], errors='coerce'), unit='s', utc=True
        ).dt.strftime('%Y-%m-%d %H:%M:%S')
        cols = ['dateTime'] + [col for col in transfers_df.columns if col != 'dateTime']
        transfers_df = transfers_df[cols]
        output_file = os.path.join(output_file, 'erc20_transfers.csv')

        transfers_df.to_csv(output_file, index=False, encoding='utf-8')
        logger.info(f"Data successfully saved to {output_file}")
    else:
        logger.warning("No valid transfers to save. DataFrame is empty.")

In [5]:
try:
    # get the block numbers
    start_block, end_block = get_block_numbers_by_date(start_date=START_DATE, end_date=END_DATE, include_all=False)
    print(start_block, end_block)
    # fetch the ERC20 transfers
    transfers_df = fetch_erc20_transfers(address=ADDRESS, start_block=start_block, end_block=end_block, offset=10)
    
    process_and_save_transfers(transfers_df, OUTPUT_FILE)

except Exception as e:
    logger.error(f"An error occurred during the execution: {e}")


2024-12-23 14:31:55,514 - INFO - Fetching ERC20 transfers from block 21368389 to 21382703...


21368389 21382703


2024-12-23 14:31:56,447 - INFO - Page 1: Retrieved 10 transactions.


{'status': '1', 'message': 'OK', 'result': [{'blockNumber': '21368861', 'timeStamp': '1733794511', 'hash': '0x7df7de1f1098529536cc4a9ee1c6500a66d18709e68542ddb15b590bab8f3088', 'nonce': '9', 'blockHash': '0xdb9c39e9e9dc86f746d0bb8a0500faf0fce171b28a391c403cc558503f595fb0', 'from': '0xe217e15b3c19cc0427f9492dc3bcfe8220afad10', 'contractAddress': '0xdac17f958d2ee523a2206206994597c13d831ec7', 'to': '0x5be9a4959308a0d0c7bc0870e319314d8d957dbb', 'value': '25541000000', 'tokenName': 'Tether USD', 'tokenSymbol': 'USDT', 'tokenDecimal': '6', 'transactionIndex': '66', 'gas': '280345', 'gasPrice': '20561899999', 'gasUsed': '171277', 'cumulativeGasUsed': '5031426', 'input': 'deprecated', 'confirmations': '94555'}, {'blockNumber': '21370001', 'timeStamp': '1733808299', 'hash': '0x0e1de3d529f9d1e488c28994f0372983e347bb448278e3b536ef7077b1654188', 'nonce': '30', 'blockHash': '0x84cbbaf1c6022b51c064303d26a2e7e1fd32d498c13692715f74ce430a51d41f', 'from': '0xe217e15b3c19cc0427f9492dc3bcfe8220afad10', 'c

2024-12-23 14:31:57,366 - INFO - Page 2: Retrieved 10 transactions.


...

{'status': '1', 'message': 'OK', 'result': [{'blockNumber': '21378518', 'timeStamp': '1733911067', 'hash': '0x9c0f87375cc734195418bb3df6d66e2e05c664e7f520726016df32a661bdb395', 'nonce': '15', 'blockHash': '0x2bdf24441df655630060fb888f82349a1d92e2f36a5728ade260b45b70db0df0', 'from': '0xe217e15b3c19cc0427f9492dc3bcfe8220afad10', 'contractAddress': '0xdac17f958d2ee523a2206206994597c13d831ec7', 'to': '0x5be9a4959308a0d0c7bc0870e319314d8d957dbb', 'value': '75000000', 'tokenName': 'Tether USD', 'tokenSymbol': 'USDT', 'tokenDecimal': '6', 'transactionIndex': '32', 'gas': '280327', 'gasPrice': '14030171586', 'gasUsed': '171265', 'cumulativeGasUsed': '6281737', 'input': 'deprecated', 'confirmations': '84898'}, {'blockNumber': '21378585', 'timeStamp': '1733911895', 'hash': '0x33a7951be688b0dd0c7fd0d5deb3cbb96806bc9258215defce1a09a314db964c', 'nonce': '1', 'blockHash': '0x217bb9c01354f1e850856785f7747043d99e194c6f8b78749465e21a7e72f7c5', 'from': '0xe217e15b3c19cc0427f9492dc3bcfe8220afad10', 'cont

2024-12-23 14:31:59,216 - INFO - Page 4: Retrieved 5 transactions.


{'status': '1', 'message': 'OK', 'result': [{'blockNumber': '21380978', 'timeStamp': '1733940791', 'hash': '0xecb57481fac2ece624e1b3d8c61a69a5b475ee19ba337130f897834cb15c49ae', 'nonce': '10514', 'blockHash': '0xc03bdf5ce77d59731711b7358a9bf79e306e4fce71d060e3197f6873f8610369', 'from': '0x5be9a4959308a0d0c7bc0870e319314d8d957dbb', 'contractAddress': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', 'to': '0x9008d19f58aabd9ed0d60971565aa8510560ab41', 'value': '2187457492438', 'tokenName': 'USDC', 'tokenSymbol': 'USDC', 'tokenDecimal': '6', 'transactionIndex': '16', 'gas': '700000', 'gasPrice': '32099595605', 'gasUsed': '200829', 'cumulativeGasUsed': '2434752', 'input': 'deprecated', 'confirmations': '82438'}, {'blockNumber': '21381100', 'timeStamp': '1733942255', 'hash': '0x7c2bd5d2017bd92a340e439e0f53d53ed390abed74126e6d8147c34277ea5ce7', 'nonce': '429', 'blockHash': '0x9fa60ffe2831952b5343250da003fade2fc2c14ceb82ae12f27a0c2f409cfb48', 'from': '0x78ff9211317620de95602c9cbed3ae803689e545', '

2024-12-23 14:32:00,167 - WARNING - API request failed on page 5: Request failed: No transactions found
2024-12-23 14:32:00,167 - INFO - Final dataset contains 35 unique transactions.
2024-12-23 14:32:00,183 - INFO - Data successfully saved to C:/Users/YuweiCao/Documents/GitHub/Project/Project/etherscan/result\erc20_transfers.csv


{'status': '0', 'message': 'No transactions found', 'result': []}
